#Q2

##a)

Given x is known, take derivative of (2) with respect to $\eta$ we can get 

$$
-2\sum_{i=1}^{n}(x_i(Cx)_i - \eta\ast)= 0
$$
$$
n\eta\ast = \sum_{i=1}^{n}x_i(Cx)_i
$$
$$
n\eta\ast = \frac{1}{n}\sum_{i=1}^{n}x_i(Cx)_i
$$
for any x in feasible set

To show uniqueness we can prove by contradiction. Assume there exit $\eta'$ not equal to $\eta\ast$ that also minimizes (2), then we have 
$$
\sum_{i=1}^{n}(x_i(Cx)_i - \eta\ast)^2 + \rho x^TCx = \sum_{i=1}^{n}(x_i(Cx)_i - \eta')^2 + \rho x^TCx
$$
$$
-2\sum_{i=1}^{n}(x_i(Cx)_i\eta\ast) + n\eta\ast^2= -2\sum_{i=1}^{n}(x_i(Cx)_i\eta') + n\eta'^2
$$

$$
(\eta\ast - \eta')(2\sum_{i=1}^{n}x_i(Cx)_i + n(\eta\ast + \eta')) = 0
$$
$\sum_{i=1}^{n}x_i(Cx)_i > 0$ since it is portfolio variance. Both $\eta\ast $ and $\eta'$ should be positive, otherwise $-\eta\ast$ and $\eta'$  are positive then (2) cannot reach the minimum point for sure. So we should have $\eta\ast - \eta' = 0 $, i.e. $\eta\ast = \eta$. So optimal $\eta$ is unique.


##b)

$$
x^T(M_i - \frac{1}{n}C)y =x^TM_iy -\frac{1}{n}Cy 
$$
plug in the constraint x=y we can get
$$
\sum_{i=1}^{n}(x^T(M_i - \overline C)y)^2 = \sum_{i=1}^{n}(x^TM_ix - \frac{1}{n}x^TCx)^2
$$
where
$$
\frac{1}{n}x^TCx = \frac{1}{n}\sum_{i=1}^{n}x_i(Cx)_i = \eta\ast 
$$

So set $x^TM_ix = x_i(Cx)_i$ we can get $M_i = CI^\ast_i$ where $I^\ast_i$ means diagonal matrix with ith diagonal entry equal to 1 and all other entries equal to zero.

When x = y, the first term of the equation (4)  $\sum_{i=1}^{n}(x^T(M_i - \overline C)x)^2 $would be convex if and only if $(M_i - \overline{C})$ is PSD. But $(M_i - \overline C)$ does not necessarily to be symmetric and $z^TM_iz - z^T\overline Cz \geq 0$ are not guaranteed for all z. So we do not have the equation (4) to be convex. 

##c)

Problem in part (b) above was non-convex, here the algorithm relaxes x=y condition and we fit optimal values of x and y in each iteration separately, and each of these steps is convex and hence can be solved using cvxpy.

The 2nd and 3rd terms of $argmin$ steps are convex due to C being PSD and norm being convex respectively. The first term is also convex because this time, $x \neq y$ and hence we are no longer dependent on $M_i - \overline{C}$ being a PSD. Overall, sum of three convex terms is convex.

In [ ]:
#c)

import numpy as np
import cvxpy as cp
import timeit


In [ ]:
def create_mlst(x):
    Mlist = []
    n = len(x)
    for i in range(n):
        M0 = np.zeros((n, n))
        M0[i][i] = 1
        Mlist.append(M0)
    return Mlist

In [ ]:
#optimization function for one interation

def opt_x(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat):
    y0 = x0
    n = len(x0)
    M_sum = 0
    Mlist = create_mlst(x0)
    
    x = cp.Variable((n,1))
    
    obj_fcn = 0
 
    for i in range(n):
        M_sum = M_sum + ((x.T @ (C@Mlist[i] - 1/n*C))@y0)**2
        
    obj_fcn = M_sum + rho*((x.T@C)@y0) + 1/miu0*cp.pnorm(y0-x,p=2)**2
    
    #constraints:
    constraints = [x>=l_bound, x<=u_bound,
                  cp.sum(x)==1,
                  r_hat.T @ x >= rho_r]
    
    f = cp.Minimize(obj_fcn)
    prob = cp.Problem(f,constraints)
    prob.solve()
    return x.value



#choose miu_d to be the percentage of how much miu decreases for every iteration
#thres represents the threshold to exit

def solve_opt(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d):
    
    n = len(x0)
    x = x0
    y = x0
    diff = 1000000
    miu = miu0
    iterations = 0
    
    while diff > thres:
        x = opt_x(y,miu,rho,rho_r,C,l_bound, u_bound,r_hat)
        y = opt_x(x,miu,rho,rho_r,C,l_bound, u_bound,r_hat)
        miu = miu*miu_d
        diff = ((x-y).T @ (x-y))**2
        iterations+=1
        
    return x,y,diff,iterations
    
    

def opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d):
    
    start = timeit.default_timer()
    
    x,y,diff,iterations = solve_opt(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
    
    stop = timeit.default_timer()
    
    # print('Optimal Weights x:', np.round(x.T,4), "\n",
    # 'L2 norm of optimal x and y:', diff,"\n",
    # 'Optimization time:', np.round(stop-start,4),"s"
    # )
    return (np.round(x.T,4),diff,np.round(stop-start,4),iterations)
    


## d)

In [ ]:
#d)
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-1]*5).reshape(-1,1)
u_bound = np.array([1]*5).reshape(-1,1)
miu0 = 1
rho = 2
rho_r = 0
thres = 0.0001
miu_d = 0.9

### Below, we are able to get an optimal value of x for given Covariance matrix



In [ ]:
optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
print('Optimal Weights x:', optimal_x, "\n",
    'L2 norm of optimal x and y:', x_y_diff,"\n",
    'Optimization time:', time_taken,"s"
    )

Optimal Weights x: [[ 0.0585 -0.0999 -0.1317  0.7082  0.4648]] 
 L2 norm of optimal x and y: [[6.17869915e-05]] 
 Optimization time: 2.4457 s


### Effect of Changing Threshold of Ending condition

Threshold of ending condition signifies how much difference between x and y is acceptable

As expected, if we decrease the threshold, it takes more iterations and more time to reach the ending condition, however after sufficiently decreasing the threshold, only a few more iterations are needed to reach ending condition, besides that time taken per iteration also drops a bit

In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-1]*5).reshape(-1,1)
u_bound = np.array([1]*5).reshape(-1,1)
miu0 = 1
rho = 2
rho_r = 0
miu_d = 0.9

for z in range(1,10):
  print('#'*40)
  thres = 10**(-z)
  print("Threshold", thres)
  optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
  print("Time Taken",time_taken, "  x_y_diff", x_y_diff, "  iterations",iterations, "time_per_iteration",np.round((time_taken/iterations),4))

########################################
Threshold 0.1
Time Taken 1.8972   x_y_diff [[0.04677998]]   iterations 24 time_per_iteration 0.079
########################################
Threshold 0.01
Time Taken 2.1283   x_y_diff [[0.00916705]]   iterations 27 time_per_iteration 0.0788
########################################
Threshold 0.001
Time Taken 2.3563   x_y_diff [[0.00085713]]   iterations 30 time_per_iteration 0.0785
########################################
Threshold 0.0001
Time Taken 2.3898   x_y_diff [[6.17869915e-05]]   iterations 32 time_per_iteration 0.0747
########################################
Threshold 1e-05
Time Taken 2.5025   x_y_diff [[9.00764354e-06]]   iterations 33 time_per_iteration 0.0758
########################################
Threshold 1e-06
Time Taken 2.6515   x_y_diff [[7.19579943e-07]]   iterations 34 time_per_iteration 0.078
########################################
Threshold 1e-07
Time Taken 2.7014   x_y_diff [[2.71570045e-08]]   iterations 35 time_per_iter

### Effect of Bounds

I increased the bounds from (-1,1) to (-10,10), Atleast this increase did not cause the number of iterations to increase, also time taken per iteration also doesnt change much, this effect of bound can be more extreme if we severely limit the upper and lower bound to be very close to each other, which will make it very difficult to reach ending conditions

In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-10]*5).reshape(-1,1)
u_bound = np.array([10]*5).reshape(-1,1)
miu0 = 1
rho = 2
rho_r = 0
miu_d = 0.9

for z in range(1,10):
  print('#'*40)
  thres = 10**(-z)
  print("Threshold", thres)
  optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
  print("Time Taken",time_taken, "  x_y_diff", x_y_diff, "  iterations",iterations, "time_per_iteration",np.round((time_taken/iterations),4))

########################################
Threshold 0.1
Time Taken 1.8077   x_y_diff [[0.07830602]]   iterations 23 time_per_iteration 0.0786
########################################
Threshold 0.01
Time Taken 2.0344   x_y_diff [[0.00931917]]   iterations 27 time_per_iteration 0.0753
########################################
Threshold 0.001
Time Taken 2.2637   x_y_diff [[0.00086176]]   iterations 30 time_per_iteration 0.0755
########################################
Threshold 0.0001
Time Taken 2.3518   x_y_diff [[6.2058482e-05]]   iterations 32 time_per_iteration 0.0735
########################################
Threshold 1e-05
Time Taken 2.4681   x_y_diff [[9.05016352e-06]]   iterations 33 time_per_iteration 0.0748
########################################
Threshold 1e-06
Time Taken 2.5605   x_y_diff [[7.2285038e-07]]   iterations 34 time_per_iteration 0.0753
########################################
Threshold 1e-07
Time Taken 2.7253   x_y_diff [[2.72735399e-08]]   iterations 35 time_per_iter

### Effect of Rho

As we increase rho from 1 to 26, we are incresing weight of $x^TCx$ term and it makes convergence of x and y a bit hard and hence it takes more iterations to reach ending condition

In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-1]*5).reshape(-1,1)
u_bound = np.array([1]*5).reshape(-1,1)
miu0 = 1
# rho = 2
rho_r = 0
miu_d = 0.9
thres = 10**(-5)

for rho in range(1,30,5):
  print('#'*40)
  print("Rho", rho)
  optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
  print("Time Taken",time_taken, "  x_y_diff", x_y_diff, "  iterations",iterations, "time_per_iteration",np.round((time_taken/iterations),4))

########################################
Rho 1
Time Taken 2.1943   x_y_diff [[3.73919063e-06]]   iterations 28 time_per_iteration 0.0784
########################################
Rho 6
Time Taken 3.6382   x_y_diff [[4.54235523e-07]]   iterations 47 time_per_iteration 0.0774
########################################
Rho 11
Time Taken 4.2739   x_y_diff [[3.80877479e-06]]   iterations 56 time_per_iteration 0.0763
########################################
Rho 16
Time Taken 4.9854   x_y_diff [[2.03980294e-06]]   iterations 64 time_per_iteration 0.0779
########################################
Rho 21
Time Taken 5.1408   x_y_diff [[7.91493108e-06]]   iterations 68 time_per_iteration 0.0756
########################################
Rho 26
Time Taken 5.5079   x_y_diff [[7.44439756e-07]]   iterations 72 time_per_iteration 0.0765


### Effect of miu_d
miu_d is the factor by which I am decreasing $\mu^{(k)}$ in each iteration, as I increase miu_d from 0.2 to 0.9, I am slowing the pace of decrease in $\mu^{(k)}$, this makes the penalty factor relevant for more iterations and makes the algorithm take more iterations to reach the ending condition of difference of x and y reaching a certain threshold


In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-1]*5).reshape(-1,1)
u_bound = np.array([1]*5).reshape(-1,1)
miu0 = 1
rho = 2
rho_r = 0
# miu_d = 0.9
thres = 10**(-5)

for miu_d_help in range(2,10):
  print('#'*40)
  miu_d = miu_d_help/10
  print("miu_d", miu_d)
  optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
  print("Time Taken",time_taken, "  x_y_diff", x_y_diff, "  iterations",iterations, "time_per_iteration",np.round((time_taken/iterations),4))

########################################
miu_d 0.2
Time Taken 0.3433   x_y_diff [[4.91752842e-09]]   iterations 4 time_per_iteration 0.0858
########################################
miu_d 0.3
Time Taken 0.4268   x_y_diff [[5.94359939e-10]]   iterations 5 time_per_iteration 0.0854
########################################
miu_d 0.4
Time Taken 0.4787   x_y_diff [[1.53491218e-09]]   iterations 6 time_per_iteration 0.0798
########################################
miu_d 0.5
Time Taken 0.5476   x_y_diff [[3.02476705e-08]]   iterations 7 time_per_iteration 0.0782
########################################
miu_d 0.6
Time Taken 0.7811   x_y_diff [[1.45986494e-08]]   iterations 9 time_per_iteration 0.0868
########################################
miu_d 0.7
Time Taken 0.9548   x_y_diff [[3.57031403e-08]]   iterations 12 time_per_iteration 0.0796
########################################
miu_d 0.8
Time Taken 1.2911   x_y_diff [[4.12071229e-06]]   iterations 17 time_per_iteration 0.0759
##################

### Effect of Profit Threshold

If we keep profit threshold fairly less or negative, we are able to reach an optimality condition of portfolio but we arent able to fit for highly positive values of profit threshold

In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])

#target return > 0 
r_hat = np.array([0]*5).reshape(-1,1) 
#start with equal weights
x0 = np.array([0.2]*5).reshape(-1,1) 
l_bound = np.array([-1]*5).reshape(-1,1)
u_bound = np.array([1]*5).reshape(-1,1)
miu0 = 1
rho = 2
rho_r = 0
miu_d = 0.9
thres = 10**(-5)

for rho_r in range(-20,40,10):
  print('#'*40)
  print("rho_r", rho_r)
  try:
    optimal_x,x_y_diff, time_taken,iterations = opt_summary(x0,miu0,rho,rho_r,C,l_bound, u_bound,r_hat, thres, miu_d)
    print("Time Taken",time_taken, "  x_y_diff", x_y_diff, "  iterations",iterations, "time_per_iteration",np.round((time_taken/iterations),4))
  except:
    print("Unable to fit")

########################################
rho_r -20
Time Taken 2.5577   x_y_diff [[9.0084002e-06]]   iterations 33 time_per_iteration 0.0775
########################################
rho_r -10
Time Taken 2.5127   x_y_diff [[9.00887763e-06]]   iterations 33 time_per_iteration 0.0761
########################################
rho_r 0
Time Taken 2.5144   x_y_diff [[9.00764354e-06]]   iterations 33 time_per_iteration 0.0762
########################################
rho_r 10
Unable to fit
########################################
rho_r 20
Unable to fit
########################################
rho_r 30
Unable to fit


### Comparison with direct solution technique

Below, I use scipy.optimize instead of Matlab's fmincon. Both y and x need to be input as a single vector and constraints, bounds are given appropriately as before using scipy's format.



In [ ]:
import scipy.optimize as optimize
import scipy.optimize as minimize
from scipy.optimize import Bounds

In [ ]:
C = np.array([[94.868,33.75,12.325,1.178,8.778],
             [33.75,445.642,98.955,7.901,84.954],
             [12.325,98.955,117.265,0.503,45.184],
             [1.178,7.901,0.503,5.46,1.057],
             [8.778,84.954,45.184,1.057,34.126]])
rho = 2

guess = np.array([0.2 for z in range(10)])

def myfunc(x):
  curr_n = int(len(x)/2)
  #print("a",curr_n)
  x1 = x[:curr_n]
  y1 = x[curr_n:]
  first_term = 0
  Mlist = create_mlst(x1)
  for z in range(curr_n):
    first_term+= (x1.T @ (C @ (Mlist[z]) - (1/curr_n)*C ) @ (y1))**2
  second_term = rho* x1.T @ (C) @ (x1)
  return (first_term+second_term)

def apply_xy_constraint(x):
    curr_n = int(len(x)/2)
    #print('b',curr_n)
    x1 = x[:curr_n]
    y1 = x[curr_n:]
    my_sum = 0
    for z in range(curr_n):
      my_sum +=abs(x1[z]-y1[z])
    return my_sum

def apply_sum_constraint(x):
    curr_n = int(len(x)/2)
    #print('c',curr_n)
    x1 = x[:curr_n]
    total = 1 - np.sum(x1)
    return total


my_bounds = Bounds([-1 for z in guess],[1 for z in guess])
my_constraints = ({'type': 'eq', "fun": apply_sum_constraint },{'type': 'eq', "fun": apply_xy_constraint })

result = optimize.minimize(myfunc, 
                      guess, 
                      method='SLSQP', 
                      # args=(a, b, c),
                      bounds=my_bounds,
                      options={'disp': True},
                      constraints=my_constraints,
                      tol = 10**(-5))
result

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 13.368177009290989
            Iterations: 81
            Function evaluations: 1069
            Gradient evaluations: 81


     fun: 13.368177009290989
     jac: array([ 36.66924977,  26.01157045,  45.13480067,  23.18568635,
        29.20314431,  -5.17817974,   5.95526946, -13.60212159,
         8.11838651,   2.22823882])
 message: 'Optimization terminated successfully.'
    nfev: 1069
     nit: 81
    njev: 81
  status: 0
 success: True
       x: array([ 0.09045647, -0.09777786,  0.04095021,  0.68276441,  0.28360677,
        0.09045584, -0.09778541,  0.04094972,  0.68276393,  0.28360747])

Optimal Weights from above contain, x as well as y vector appended together and their values match as expected.

These direct solution(above) values are also pretty close to our encoded relaxation approach values(below) as expected.

[[ 0.0585 -0.0999 -0.1317  0.7082  0.4648]] 